In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import utils.data_preparation as prep
import utils.prep_info as prep_info
import utils.prep_measurements as prep_measurements

In [3]:
# Input Variables
dir_path = "Textfiles"

In [4]:
def get_experiment_measurements_df(dir_path, file_name, trim = 25):
    
    # Combine directory path and file name to get the full file path
    file_path = os.path.join(dir_path, file_name)
    
    # Split the data into info and measurements
    info_text, measurements_text = prep.split_text(file_path)

    # Process the info text into a DataFrame
    info_df = prep_info.get_info_df(info_text)

    # Process the measurements text into a DataFrame
    measurements_df = prep_measurements.get_measurement_df(measurements_text, info_df, trim)
    
    # Ensure Datatype 
    measurements_df[measurements_df.columns] = measurements_df.apply(lambda col: pd.to_numeric(col, errors='coerce'))

    return measurements_df

In [5]:
def get_monotonicity(df, interval_size=20, start_temp=-25):
    """
    Bestimmt die Monotonie (steigend, fallend, konstant) für jeden Intervallbereich.
    
    Args:
        df (pd.DataFrame): DataFrame mit Temperaturwerten als Index und Messwerten als Spalten.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: DataFrame mit der Monotonie pro Intervall.
    """
    # Berechnung des Endwerts basierend auf dem maximalen Temperaturwert
    end_temp = int(df.index.max() // interval_size * interval_size + interval_size)
    
    # Intervalle definieren
    interval_bins = pd.interval_range(start=start_temp, end=end_temp, freq=interval_size, closed='left')
    df['Interval'] = pd.cut(df.index, bins=interval_bins)
    
    # Funktion zur Bestimmung der Monotonie
    def calculate_monotonicity(group):
        if len(group) < 2:  # Mindestens zwei Werte erforderlich
            return pd.Series({col: None for col in df.columns if col != 'Interval'})
        x = group.index.astype(float)  # Temperaturwerte (x-Achse) als float sicherstellen
        results = {}
        for col in df.columns:
            if col == 'Interval':  # Skip Interval column
                continue
            y = group[col].values  # Messwerte (y-Achse)
            gradient = np.gradient(y, x).mean()  # Durchschnittsgradient für das Intervall
            
            # Monotonie basierend auf der Steigung bestimmen
            if gradient > 0:
                trend = "steigend"
            elif gradient < 0:
                trend = "fallend"
            else:
                trend = "konstant"
            
            results[col] = trend
        return pd.Series(results)
    
    # Monotonie für jede Gruppe berechnen
    monotonicity_df = df.groupby('Interval', observed=False).apply(calculate_monotonicity)
    
    return monotonicity_df

In [6]:
def get_single_monotonicity_df(dir_path, file_name, interval_size=20, start_temp=-25):
    """
    Verarbeitet eine experimentelle Datei und gibt die Monotonie (steigend, fallend, konstant) zurück.
    
    Args:
        dir_path (str): Verzeichnis, in dem die Datei gespeichert ist.
        file_name (str): Name der Datei, die verarbeitet werden soll.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: DataFrame mit der Monotonie pro Intervall.
    """
    # Lese die experimentellen Messdaten ein
    measurements_df = get_experiment_measurements_df(dir_path, file_name)
    
    # Berechne die Monotonie
    monotonicity_df = get_monotonicity(measurements_df, interval_size=interval_size, start_temp=start_temp)
    
    return monotonicity_df

In [7]:
def get_merged_monotonicity_df(dir_path, interval_size=20, start_temp=-25):
    """
    Verarbeitet alle Dateien in einem Verzeichnis und erstellt einen zusammengeführten Monotonie-DataFrame.
    
    Args:
        dir_path (str): Verzeichnis mit den Dateien.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: Zusammengeführter DataFrame mit Monotonie-Daten für alle Dateien.
    """
    merged_df = None  # Initialisiere den zusammengeführten DataFrame
    
    # Iteriere über alle Dateien im Verzeichnis
    for file_name in os.listdir(dir_path):
        if file_name.endswith(".txt"):  # Nur Textdateien verarbeiten
            try:
                # Monotonie-DataFrame für die aktuelle Datei
                monotonicity_df = get_single_monotonicity_df(
                    dir_path, file_name, interval_size=interval_size, start_temp=start_temp)
                
                # Füge die Ergebnisse zum zusammengeführten DataFrame hinzu
                if merged_df is None:
                    merged_df = monotonicity_df
                else:
                    merged_df = pd.concat([merged_df, monotonicity_df], axis=1)
            
            except Exception as e:
                print(e)
    return merged_df

### 1) Monotonie 

In [9]:
merged_monotonicity_df = get_merged_monotonicity_df(dir_path, interval_size=20, start_temp=-25)
merged_monotonicity_df

C:\Users\konra\AppData\Local\Temp\ipykernel_20668\4253433860.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monotonicity_df = df.groupby('Interval', observed=False).apply(calculate_monotonicity)
C:\Users\konra\AppData\Local\Temp\ipykernel_20668\4253433860.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monotonicity_df = df.groupby('Interval', observed=False).apply(calculate_monotonicity)
C:\Users

,L001_S3,L001_S5,L002_S3,L002_S5,L003_S3,L003_S5,L004_S3,L004_S5,L005_S3,L005_S5,...,L383_S3,L383_S5,L384_S3,L384_S5,L385_S3,L385_S5,L386_S3,L386_S5,L387_S3,L387_S5
Interval,,,,,,,,,,,,,,,,,,,,,
"[-25, -5)",fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[-5, 15)",fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[15, 35)",fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[35, 55)",steigend,steigend,fallend,steigend,steigend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[55, 75)",steigend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[75, 95)",steigend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[95, 115)",fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,...,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend
"[115, 135)",steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,steigend,fallend,...,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
"[135, 155)",steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,...,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,fallend,steigend


In [10]:
# Transponiere den DataFrame, um die Zeilen und Spalten zu tauschen
transposed_df = merged_monotonicity_df.T

# Index-Bezeichnung ändern
transposed_df.index.name = "Probe"

# Spalten-Bezeichnung optional ändern
#transposed_df.columns.name = "Intervall"

transposed_df = transposed_df.add_prefix("Mon_")
transposed_df

Interval,"Mon_[-25, -5)","Mon_[-5, 15)","Mon_[15, 35)","Mon_[35, 55)","Mon_[55, 75)","Mon_[75, 95)","Mon_[95, 115)","Mon_[115, 135)","Mon_[135, 155)","Mon_[155, 175)","Mon_[175, 195)","Mon_[195, 215)","Mon_[215, 235)","Mon_[235, 255)","Mon_[255, 275)"
Probe,,,,,,,,,,,,,,,
L001_S3,fallend,fallend,fallend,steigend,steigend,steigend,fallend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
L001_S5,steigend,steigend,steigend,steigend,steigend,steigend,steigend,fallend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
L002_S3,fallend,fallend,fallend,fallend,fallend,fallend,fallend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
L002_S5,steigend,steigend,steigend,steigend,steigend,steigend,steigend,fallend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
L003_S3,fallend,fallend,fallend,steigend,fallend,fallend,fallend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L385_S5,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend
L386_S3,fallend,fallend,fallend,fallend,fallend,fallend,fallend,steigend,steigend,fallend,steigend,steigend,fallend,steigend,steigend
L386_S5,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend,steigend


### 2) Steigung

In [12]:
def get_slope_values_df(df, interval_size=20, start_temp=-25):
    """
    Berechnet die Steigung (Gradient) für jeden Intervallbereich basierend auf festen Intervallgrenzen.
    
    Args:
        df (pd.DataFrame): DataFrame mit Temperaturwerten als Index und Messwerten als Spalten.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: DataFrame mit Steigungen (Gradienten) pro Intervall.
    """
    # Berechnung des Endwerts basierend auf dem maximalen Temperaturwert
    end_temp = int(df.index.max() // interval_size * interval_size + interval_size)
    
    # Intervalle definieren
    interval_bins = pd.interval_range(start=start_temp, end=end_temp, freq=interval_size, closed='left')
    df['Interval'] = pd.cut(df.index, bins=interval_bins)
    
    # Funktion zur Berechnung des Gradienten (Steigung) für jede Spalte
    def calculate_gradient(group):
        if len(group) < 2:  # Mindestens zwei Werte erforderlich
            return pd.Series({col: None for col in df.columns})
        x = group.index.astype(float)  # Temperaturwerte (x-Achse) als float sicherstellen
        gradients = {}
        for col in df.columns:
            if col == 'Interval':  # Skip Interval column
                continue
            y = group[col].values  # Messwerte (y-Achse)
            gradient = np.gradient(y, x)  # Gradient berechnen
            gradients[col] = gradient.mean()  # Durchschnittsgradient für das Intervall
        return pd.Series(gradients)
    
    # Gradient (Steigung) für jede Gruppe berechnen
    slope_values = df.groupby('Interval', observed=False).apply(calculate_gradient)
    
    return slope_values

In [13]:
def get_single_slope_df(dir_path, file_name, interval_size=20, start_temp=-25):
    """
    Verarbeitet eine experimentelle Datei und berechnet die Steigungen (Gradienten) für jeden Intervallbereich.
    
    Args:
        dir_path (str): Verzeichnis, in dem die Datei gespeichert ist.
        file_name (str): Name der Datei, die verarbeitet werden soll.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: DataFrame mit Steigungswerten (Gradienten) pro Intervall.
    """
    # Lese die experimentellen Messdaten ein
    measurements_df = get_experiment_measurements_df(dir_path, file_name)
    
    # Berechne die Steigungen
    slope_df = get_slope_values_df(measurements_df, interval_size=interval_size, start_temp=start_temp)
    
    return slope_df

In [14]:
def get_merged_slope_df(dir_path, interval_size=20, start_temp=-25):
    """
    Verarbeitet alle Dateien in einem Verzeichnis und erstellt einen zusammengeführten DataFrame mit Steigungswerten.
    
    Args:
        dir_path (str): Verzeichnis mit den Dateien.
        interval_size (int): Größe der Intervalle in Grad Celsius (Standard: 20).
        start_temp (int): Startwert des ersten Intervalls (Standard: -25).
        
    Returns:
        pd.DataFrame: Zusammengeführter DataFrame mit Steigungswerten (Gradienten) für alle Dateien.
    """
    merged_df = None  # Initialisiere den zusammengeführten DataFrame
    
    # Iteriere über alle Dateien im Verzeichnis
    for file_name in os.listdir(dir_path):
        if file_name.endswith(".txt"):  # Nur Textdateien verarbeiten
            try:
                # Steigungs-DataFrame für die aktuelle Datei
                slope_df = get_single_slope_df(
                    dir_path, file_name, interval_size=interval_size, start_temp=start_temp)
                
                # Füge die Ergebnisse zum zusammengeführten DataFrame hinzu
                if merged_df is None:
                    merged_df = slope_df
                else:
                    merged_df = pd.concat([merged_df, slope_df], axis=1)
            
            except Exception as e:
                print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    
    return merged_df

In [15]:
get_merged_slope_df("Textfiles")

C:\Users\konra\AppData\Local\Temp\ipykernel_20668\3215004857.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slope_values = df.groupby('Interval', observed=False).apply(calculate_gradient)
C:\Users\konra\AppData\Local\Temp\ipykernel_20668\3215004857.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slope_values = df.groupby('Interval', observed=False).apply(calculate_gradient)
C:\Users\konra\AppData

,L001_S3,L001_S5,L002_S3,L002_S5,L003_S3,L003_S5,L004_S3,L004_S5,L005_S3,L005_S5,...,L383_S3,L383_S5,L384_S3,L384_S5,L385_S3,L385_S5,L386_S3,L386_S5,L387_S3,L387_S5
Interval,,,,,,,,,,,,,,,,,,,,,
"[-25, -5)",-1.635000e-04,0.000697,-0.000147,0.000664,-0.000044,0.000703,-0.000433,0.000573,-0.000167,0.000693,...,-0.000500,0.000700,-0.000509,0.000802,-0.000519,0.000669,-0.000557,0.000740,-0.000537,0.000798
"[-5, 15)",-7.050000e-05,0.000519,-0.000137,0.000471,-0.000075,0.000427,-0.000328,0.000379,-0.000101,0.000388,...,-0.000263,0.000576,-0.000180,0.000671,-0.000294,0.000584,-0.000297,0.000572,-0.000247,0.000635
"[15, 35)",-1.900000e-05,0.000543,-0.000064,0.000535,-0.000021,0.000541,-0.000239,0.000527,-0.000149,0.000611,...,-0.000251,0.000549,-0.000211,0.000728,-0.000245,0.000565,-0.000210,0.000599,-0.000211,0.000606
"[35, 55)",2.975000e-05,0.000484,-0.000039,0.000472,0.000028,0.000456,-0.000108,0.000413,-0.000035,0.000526,...,-0.000214,0.000506,-0.000123,0.000642,-0.000232,0.000577,-0.000224,0.000590,-0.000207,0.000634
"[55, 75)",3.650000e-05,0.000433,-0.000016,0.000402,-0.000006,0.000375,-0.000112,0.000319,-0.000094,0.000466,...,-0.000234,0.000526,-0.000115,0.000668,-0.000304,0.000580,-0.000226,0.000629,-0.000225,0.000654
"[75, 95)",7.500000e-07,0.000442,-0.000039,0.000435,-0.000017,0.000413,-0.000064,0.000419,-0.000072,0.000538,...,-0.000186,0.000580,-0.000065,0.000697,-0.000217,0.000618,-0.000212,0.000658,-0.000178,0.000735
"[95, 115)",-1.580000e-03,0.000868,-0.001231,0.000765,-0.001330,0.000827,-0.001092,0.000743,-0.001484,0.000912,...,-0.000721,0.000683,-0.001032,0.000895,-0.001035,0.000826,-0.001039,0.000889,-0.000983,0.000929
"[115, 135)",2.266500e-03,-0.000630,0.001880,-0.000431,0.001926,-0.000550,0.001610,-0.000330,0.002131,-0.000625,...,0.000617,0.000440,0.001323,0.000636,0.000789,0.000266,0.000963,0.000315,0.001194,0.000373
"[135, 155)",2.907500e-04,0.000358,0.000244,0.000403,0.000249,0.000388,0.000210,0.000420,0.000248,0.000429,...,0.000033,0.000557,0.000219,0.000813,0.000035,0.000629,0.000154,0.000680,-0.000270,0.000842


### 3) Einfache Import aller Werte

In [17]:
def round_and_average_index(df):
    """
    Rundet Float-Werte des Index auf Integer ab und bildet den Durchschnitt für doppelte Indexwerte.
    
    Args:
        df (pd.DataFrame): DataFrame mit einem Float-Index.
        
    Returns:
        pd.DataFrame: DataFrame mit Integer-Index, wobei doppelte Indexwerte aggregiert (gemittelt) wurden.
    """
    # Float-Index in Integer umwandeln
    #df.index = df.index.astype(int)
    df.index = df.index.round(0).astype(int)
    
    # Gruppieren und den Durchschnitt für doppelte Indexwerte berechnen
    df = df.groupby(df.index).mean()
    
    return df

def get_merged_experiment_measurements_df(dir_path, trim=25):
    """
    Verarbeitet alle Dateien in einem Verzeichnis und erstellt einen zusammengeführten DataFrame,
    indem Spalten aus den Dateien hinzugefügt werden.
    
    Args:
        dir_path (str): Verzeichnis, in dem die Dateien gespeichert sind.
        
    Returns:
        pd.DataFrame: Zusammengeführter DataFrame mit experimentellen Messwerten für alle Dateien.
    """
    merged_df = pd.DataFrame()  # Initialisiere leeren DataFrame
    
    # Iteriere über alle Dateien im Verzeichnis
    for file_name in os.listdir(dir_path):
        if file_name.endswith(".txt"):  # Nur Textdateien verarbeiten
            try:
                # Messwerte-DataFrame für die aktuelle Datei
                measurements_df = get_experiment_measurements_df(dir_path, file_name, trim)
                
                # Float-Index auf Integer abrunden und Mehrfachwerte aggregieren
                measurements_df = round_and_average_index(measurements_df)
                
                # Eindeutige Benennung der Spalten für diese Datei
                #measurements_df.columns = [f"{file_name}_{col}" for col in measurements_df.columns]
                
                # Füge die Ergebnisse als neue Spalten hinzu
                merged_df = pd.concat([merged_df, measurements_df], axis=1)
            
            except Exception as e:
                print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
                
    # DataFrame nach Index sortieren
    merged_df = merged_df.sort_index()
    
    return merged_df


In [18]:
df_experiments = get_merged_experiment_measurements_df(dir_path, trim=25)
df_experiments

,L001_S3,L001_S5,L002_S3,L002_S5,L003_S3,L003_S5,L004_S3,L004_S5,L005_S3,L005_S5,...,L383_S3,L383_S5,L384_S3,L384_S5,L385_S3,L385_S5,L386_S3,L386_S5,L387_S3,L387_S5
Temp./°C,,,,,,,,,,,,,,,,,,,,,
-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-25,-0.21969,0.16532,-0.20393,0.15532,-0.21751,0.15823,-0.19859,0.16202,-0.22759,0.17665,...,-0.17991,0.17382,-0.21264,0.19216,-0.19108,0.17968,-0.20179,0.19162,-0.20837,0.19495
-24,-0.21989,0.16598,-0.20426,0.15589,-0.21713,0.15900,-0.19927,0.16244,-0.22834,0.17739,...,-0.18067,0.17461,-0.21344,0.19298,-0.19186,0.18028,-0.20261,0.19231,-0.20920,0.19564
-23,-0.22134,0.16647,-0.20408,0.15648,-0.21732,0.15971,-0.20029,0.16288,-0.22837,0.17797,...,-0.18135,0.17530,-0.21442,0.19361,-0.19282,0.18083,-0.20349,0.19302,-0.21002,0.19661
-22,-0.22069,0.16703,-0.20420,0.15707,-0.21706,0.16041,-0.20056,0.16350,-0.22794,0.17882,...,-0.18193,0.17578,-0.21536,0.19430,-0.19367,0.18136,-0.20438,0.19379,-0.21093,0.19738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,-0.14694,0.30134,-0.14925,0.28520,-0.15352,0.29349,-0.16734,0.29197,-0.17898,0.31313,...,-0.20251,0.34632,-0.19537,0.42980,-0.21137,0.35797,-0.21101,0.39123,-0.21037,0.41242
273,-0.14645,0.30176,-0.15095,0.28576,-0.15315,0.29423,-0.16823,0.29250,-0.17744,0.31342,...,-0.19908,0.34696,-0.19620,0.43046,-0.21059,0.35851,-0.20958,0.39171,-0.21226,0.41305
274,-0.14583,0.30208,-0.14905,0.28632,-0.15369,0.29483,-0.16664,0.29303,-0.17628,0.31377,...,-0.20131,0.34756,-0.19337,0.43096,-0.21092,0.35839,-0.21055,0.39236,-0.20964,0.41416


In [19]:
df_cleaned = df_experiments.dropna(axis=0, how='any')
df_cleaned

,L001_S3,L001_S5,L002_S3,L002_S5,L003_S3,L003_S5,L004_S3,L004_S5,L005_S3,L005_S5,...,L383_S3,L383_S5,L384_S3,L384_S5,L385_S3,L385_S5,L386_S3,L386_S5,L387_S3,L387_S5
Temp./°C,,,,,,,,,,,,,,,,,,,,,
-24,-0.21989,0.16598,-0.20426,0.15589,-0.21713,0.15900,-0.19927,0.16244,-0.22834,0.17739,...,-0.18067,0.17461,-0.21344,0.19298,-0.19186,0.18028,-0.20261,0.19231,-0.20920,0.19564
-23,-0.22134,0.16647,-0.20408,0.15648,-0.21732,0.15971,-0.20029,0.16288,-0.22837,0.17797,...,-0.18135,0.17530,-0.21442,0.19361,-0.19282,0.18083,-0.20349,0.19302,-0.21002,0.19661
-22,-0.22069,0.16703,-0.20420,0.15707,-0.21706,0.16041,-0.20056,0.16350,-0.22794,0.17882,...,-0.18193,0.17578,-0.21536,0.19430,-0.19367,0.18136,-0.20438,0.19379,-0.21093,0.19738
-21,-0.22040,0.16769,-0.20415,0.15773,-0.21713,0.16099,-0.20061,0.16419,-0.22836,0.17962,...,-0.18263,0.17655,-0.21599,0.19511,-0.19413,0.18202,-0.20510,0.19448,-0.21164,0.19795
-20,-0.22059,0.16849,-0.20421,0.15857,-0.21691,0.16159,-0.20101,0.16484,-0.22840,0.18030,...,-0.18334,0.17710,-0.21663,0.19593,-0.19459,0.18269,-0.20588,0.19512,-0.21252,0.19866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,-0.14800,0.30041,-0.15145,0.28398,-0.15596,0.29227,-0.16994,0.29114,-0.17889,0.31237,...,-0.20089,0.34542,-0.19769,0.42751,-0.21116,0.35646,-0.21207,0.38997,-0.21226,0.41070
271,-0.14738,0.30087,-0.14924,0.28462,-0.15565,0.29285,-0.16835,0.29161,-0.17823,0.31277,...,-0.20402,0.34589,-0.19540,0.42880,-0.21123,0.35731,-0.21093,0.39048,-0.21260,0.41161
272,-0.14694,0.30134,-0.14925,0.28520,-0.15352,0.29349,-0.16734,0.29197,-0.17898,0.31313,...,-0.20251,0.34632,-0.19537,0.42980,-0.21137,0.35797,-0.21101,0.39123,-0.21037,0.41242


In [20]:
# Speichern des DataFrames als CSV-Datei mit Index
df_cleaned.to_csv("data/02_dsc_thermogram.csv", index=True)